In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gym
import wandb
import numpy as np
from tqdm import tqdm

class Policy(nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(Policy, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)

class Value(nn.Module):
    
    #Takes in state
    def __init__(self, observation_space):
        super(Value, self).__init__()
        
        self.input_layer = nn.Linear(observation_space, 128)
        self.output_layer = nn.Linear(128, 1)
        
    def forward(self, x):
        #input layer
        x = self.input_layer(x)
        
        #activiation relu
        x = F.relu(x)
        
        #get state value
        state_value = self.output_layer(x)
        
        return state_value
    
class REINFORCE:
    def __init__(self, state_dim, hidden_dim, action_dim, learning_rate, gamma,
                 device,v_lr):
        self.policy_net = Policy(state_dim, hidden_dim, action_dim).to(device)
        self.value_net=Value(state_dim).to(device)
        self.optimizer = torch.optim.Adam(self.policy_net.parameters(),
                                          lr=learning_rate)
        self.voptimizer=torch.optim.Adam(self.value_net.parameters(),lr=v_lr)
        self.gamma = gamma
        self.device = device

    def take_action(self, state):  # randomly sample an action according to the probability distribution
        state = torch.tensor(state.reshape(1, -1), dtype=torch.float).to(self.device) 
                                                                      
        probs = self.policy_net(state)
        action_dist = torch.distributions.Categorical(probs) 
        action = action_dist.sample()
        return action.item()
    

    def update(self, transition_dict):
        reward_list = transition_dict['rewards']
        state_list = transition_dict['states']
        action_list = transition_dict['actions']

        G = 0
        self.optimizer.zero_grad()
        self.voptimizer.zero_grad()
        for i in reversed(range(len(reward_list))):  # Back to Front, calc G
            reward = reward_list[i]
            state = torch.tensor(state_list[i].reshape(1, -1),
                                 dtype=torch.float).to(self.device)
            action = torch.tensor(action_list[i]).view(-1, 1).to(self.device)
            log_prob = torch.log(self.policy_net(state).gather(1, action))  # calc log probability of action
            G = self.gamma * G + reward  # calculate return starting from this step
            state_value=self.value_net(state)
            loss = -log_prob * (G - state_value)    # loss of each step
            loss.backward(retain_graph=True) # The gradient will be accumulated in each step
            
        self.optimizer.step()
        target_value = torch.tensor([[G]], dtype=torch.float).to(self.device)  # Target value is the calculated return G

        # Calculate the Mean Squared Error loss
        value_loss = F.mse_loss(state_value, target_value)

        # Backpropagate and update the value network
        value_loss.backward(retain_graph=True)
        self.voptimizer.step()




In [2]:

# agent params
learning_rate = 1e-3
gamma = 0.99
hidden_dim = 128
v_lr=0.005
# training params
num_pbar = 10
num_episodes = 500
num_seeds = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# environment params
env_name = "CartPole-v1"


# Define the sweep configuration
sweep_config = {
    "method": "bayes",
    "metric": {"name": "regret", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"min": 1e-5, "max": 1e-2},
        "v_lr": {"min": 1e-5, "max": 1e-2},
        "hidden_dim": {"values": [64, 128, 256]}
    },
    "project": "Cartpole_with_baseline",
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 3,
        "max_iter": 100
}
}
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config)

def train(learning_rate,v_lr,hidden_dim):
    # create environment
    env = gym.make(env_name)
    env.reset(seed=0)
    # create agent
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    agent = REINFORCE(state_dim, hidden_dim, action_dim, learning_rate, gamma, device,v_lr)
    # record episode return, for plotting
    return_list = []
    # train agent
    for i_episode in range(num_episodes):
        episode_return = 0 # return of the whole episode
        transition_dict = { # store transitions of the episode
            "states": [],
            "actions": [],
            "next_states": [],
            "rewards": [],
            "dones": []
        }
        state, _ = env.reset() # reset environment
        terminated, truncated = False, False
        while not terminated and not truncated:
            action = agent.take_action(state) # select an action
            next_state, reward, terminated, truncated, _ = env.step(action) # take action and get next state, reward, termination signal
            # store transition
            transition_dict["states"].append(state)
            transition_dict["actions"].append(action)
            transition_dict["next_states"].append(next_state)
            transition_dict["rewards"].append(reward)
            transition_dict["dones"].append(terminated)
            state = next_state
            episode_return += reward
        # add episode return to return list
        return_list.append(episode_return)
        # update agent per episode(MC)
        agent.update(transition_dict)

    avg = []
    for i in range(num_episodes):
        if i < 100:
            avg_return = np.mean(return_list[:i+1])
        else:
            avg_return = np.mean(return_list[i-100:i+1])
        avg.append(avg_return)

    regret = 0
    for i in avg:
        if i < 195:
            regret += 195 - i

    return regret    
   
def run_training():
    config_defaults = {
        "learning_rate": 1e-3,
        "v_lr": 0.005,
        "hidden_dim": 128
    }
    config = wandb.init(config=config_defaults,project="baseline_cartpole")
    learning_rate = config.config["learning_rate"]
    v_lr=config.config['v_lr']
    hidden_dim = config.config["hidden_dim"]
    regret = train(learning_rate,v_lr, hidden_dim)
    wandb.log({"regret": regret})

# Run the sweep
wandb.agent(sweep_id, function=run_training)




Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 6w1s5fx9
Sweep URL: https://wandb.ai/rl_shobhith/Cartpole_with_baseline/sweeps/6w1s5fx9


wandb: Agent Starting Run: 5fevi73q with config:
wandb: 	hidden_dim: 128
wandb: 	learning_rate: 0.003196211463694309
wandb: 	v_lr: 0.0072537655215280045
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shobhith-v (rl_shobhith). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


C:\Users\Shobhith Vadlamudi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


regret,▁
regret,61274.67678


...

regret,▁
regret,69423.54112


wandb: Agent Starting Run: wa93fcdd with config:
wandb: 	hidden_dim: 64
wandb: 	learning_rate: 0.007218868063863884
wandb: 	v_lr: 0.0004898065893319532
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


regret,▁
regret,29577.09201


wandb: Agent Starting Run: qqn02qa0 with config:
wandb: 	hidden_dim: 256
wandb: 	learning_rate: 0.0012707675996378625
wandb: 	v_lr: 0.0013400570927232765
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


: 